In [1]:
%reset -fs

In [2]:
import pandas as pd
import numpy as np
import time, os
from bs4 import BeautifulSoup
import datetime
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from pymongo import MongoClient
from dotenv import load_dotenv
pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', None)

In [3]:
load_dotenv()

True

In [4]:
mongo_pw = os.environ.get("MONGO_PW")

In [5]:
client = MongoClient('mongodb+srv://natedirenzo:' + mongo_pw + '@coolkidscluster.grcwg.mongodb.net/myFirstDatabase?retryWrites=true&w=majority')


In [6]:
client.list_database_names()

['admin', 'local']

In [7]:
base = datetime.datetime.today()
delta = base - datetime.datetime(2021, 10, 19)
date_list = [base - datetime.timedelta(days=x) for x in range(delta.days+1)]
date_list = [d.strftime('%Y-%m-%d') for d in date_list]

In [8]:
#run this initially; then don't run again if you just want to keep adding to it
final_df = pd.DataFrame()

In [9]:
bet_search =  'https://www.bettingpros.com/nba/picks/prop-bets/bet/points/?date='

In [10]:
chromedriver = "/Applications/chromedriver"
os.environ['webdriver.chrome.driver'] = chromedriver

In [11]:
driver = webdriver.Chrome(chromedriver)

In [12]:
# Tip of the hat to Rita for helping with these two functions !
def get_site_rb(date):
    bet_query = bet_search + date
    driver.get(bet_query)
    time.sleep(5)
    try:
        element = driver.find_element_by_xpath('//*[@id="onetrust-accept-btn-handler"]')
        action = ActionChains(driver)
        action.move_to_element(element).click().perform()
    except:
        pass
    get_data_rb()


In [13]:
def get_data_rb():
    data_str = []
    data = []
    li = []
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    table = soup.find('table')
    date = soup.find('h1').text
    date = date.split(" ")[5]
    date.strip(')')
    date = date.replace('(',"").replace(')',"")
    house_accuracy = soup.find('div', class_='best-props__accuracy').text
    table_data = table.tbody.find_all('tr')
    for tr in table_data:
        data_str.append(tr.text.replace('\n', ' ').strip())
    for i in data_str:
        li = list(i.split('            '))
        data.append(li)
    for i in data:
        if len(i) == 7:
            i.append('TBD')
            i.append(date)
            i.append(house_accuracy)
        else:
            i.append(date)
            i.append(house_accuracy)
    global final_df
    final_df = pd.concat([final_df, pd.DataFrame(data)])
    time.sleep(1)

    try:
        page_progress = driver.find_element_by_xpath('//div[contains(@class, "table__pagination")]').text.strip("«").strip("»").strip()
        if int(page_progress[5]) == int(page_progress[10]):
            return final_df
        else:
            element = driver.find_element_by_xpath('/html/body/div[2]/main/div/div/div/div[4]/div/div/a[2]')
            action = ActionChains(driver)
            action.move_to_element(element).click().perform()
            time.sleep(3)
            get_data_rb()
    except:
        return final_df


In [14]:
#run this and then close the cookies pop-up! I really need to make a better error handler for this.
for i in date_list:
    
    if i == '2021-12-24': #Christmas Eve
        continue
    if i == '2021-11-25': #Thanksgiving
        continue
    if i == '2022-02-18': #MLK
        continue
    if i == '2022-02-19': #All Star Break
        continue
    if i == '2022-02-20': #All Star Break
        continue
    if i == '2022-02-21': #All Star Break
        continue
    if i == '2022-02-22': #All Star Break
        continue
    if i == '2022-02-23': #All Star Break
        continue
    if i == '2022-04-04': #NCAA Championship
        continue
    get_site_rb(i)

In [15]:
driver.close()

In [16]:
#Clean up below
final_df.columns=['Name', 'Team', 'Opp', 'Type', 'Line', 'Pick', 'Moneyline', 'Result', 'Date', 'Accuracy']

In [17]:
final_df.reset_index(drop=True, inplace=True)

In [18]:
final_df

,Name,Team,Opp,Type,Line,Pick,Moneyline,Result,Date,Accuracy
0,1 J. Jalen Green,HOU - G,@ BKN,Points,24.50 (-115)18.90Diff: -5.60 Line: 24.5...,24.5,-115 BetRivers,TBD,04-05-2022,Last 7 days Top 15 Accuracy: 39-50 (44%) Sea...
1,2 T. Theo Maledon,OKC - G,vs. POR,Points,17.50 (-113)12.60Diff: -4.90 Line: 17.5...,17.5,-113 BetRivers,TBD,04-05-2022,Last 7 days Top 15 Accuracy: 39-50 (44%) Sea...
2,3 K. Kevin Love,CLE - F,@ ORL,Points,11.50 (-115)16.30Diff: 4.80 Line: 11.50...,11.5,-115 PointsBet,TBD,04-05-2022,Last 7 days Top 15 Accuracy: 39-50 (44%) Sea...
3,4 K. Keon Johnson,POR - G,@ OKC,Points,13.50 (-105)9.00Diff: -4.50 Line: 13.50...,13.5,-105 PointsBet,TBD,04-05-2022,Last 7 days Top 15 Accuracy: 39-50 (44%) Sea...
4,5 K. Keldon Johnson,SAS - F,@ DEN,Points,21.50 (-135)17.60Diff: -3.90 Line: 21.5...,21.5,-135 PointsBet,TBD,04-05-2022,Last 7 days Top 15 Accuracy: 39-50 (44%) Sea...
...,...,...,...,...,...,...,...,...,...,...
14867,22 G. Grayson Allen,MIL - G,vs. BKN,Points,10.50 (-113)11.23Diff: 0.73 Line: 10.50...,10.5,-113 SugarHouse 10,TBD,10-19-2021,Season-to-date Top 15 Accuracy: 1174-1140 (5...
14868,23 N. Nemanja Bjelica,GSW - F,@ LAL,Points,6.50 (-120)6.87Diff: 0.37 Line: 6.50 Co...,6.5,-120 bet365 15,TBD,10-19-2021,Season-to-date Top 15 Accuracy: 1174-1140 (5...
14869,24 D. DeAndre Jordan,LAL - C,vs. GSW,Points,4.50 (-130)4.82Diff: 0.32 Line: 4.50 Co...,4.5,-130 bet365 2,TBD,10-19-2021,Season-to-date Top 15 Accuracy: 1174-1140 (5...
14870,25 P. Pat Connaughton,MIL - G,vs. BKN,Points,7.50 (-111)7.19Diff: -0.31 Line: 7.50 C...,7.5,-111 BetMGM 20,TBD,10-19-2021,Season-to-date Top 15 Accuracy: 1174-1140 (5...


In [19]:
player_name = final_df.Name.str.split(" ", expand = True)
final_df['Name'] = player_name[2] + ' ' + player_name[4]

In [20]:
team = final_df.Team.str.split(" ", expand = True)
final_df['Team'] = team[0]
final_df['Position'] = team[2]

In [21]:
opp = final_df.Opp.str.split(" ", expand = True)
final_df.Opp = opp[1]
final_df['Home/Away'] = opp[0]

In [22]:
bet_info = final_df.Line.str.split(" ", expand = True)

In [23]:
money_line = final_df.Moneyline.str.split(" ", expand = True);money_line

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,,,-115,BetRivers,None,None,None,None,None,None,None,None,None,None,None
1,,,-113,BetRivers,None,None,None,None,None,None,None,None,None,None,None
2,,,-115,PointsBet,None,None,None,None,None,None,None,None,None,None,None
3,,,-105,PointsBet,None,None,None,None,None,None,None,None,None,None,None
4,,,-135,PointsBet,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14867,,,-113,SugarHouse,,,,,,,,,,,10
14868,,,-120,bet365,,,,,,,,,,,15
14869,,,-130,bet365,,,,,,,,,,,2
14870,,,-111,BetMGM,,,,,,,,,,,20


In [24]:
money_line[3].value_counts()

BetMGM         5186
SugarHouse     4310
bet365         3315
PointsBet      1958
UniBet           44
BetRivers        38
PartyCasino      17
FanDuel           4
Name: 3, dtype: int64

In [25]:
for i in money_line[3]:
    for j in final_df['Result']:
        if i == None:
            continue
        elif i.isdigit() == True and j == 'TBD':
            j == i
        else:
            continue

In [26]:
final_df['Line'] = bet_info[7]
final_df['Projection'] = bet_info[15]
final_df['Diff'] = bet_info[9]
final_df['Moneyline'] = bet_info[13]
if len(bet_info[bet_info[16] == '']) > 0:
    final_df['over_under'] = bet_info[19]
else:
    final_df['over_under'] = bet_info[16]
final_df['over_under'] = final_df['over_under'].fillna('o')

In [27]:
final_df = final_df[['Date', 'Name', 'Team', 'Position', 'Opp','Home/Away', 'Line', 'Moneyline', 'Projection', 'Diff', 'Pick', 'Result', 'Accuracy', 'over_under']]

In [28]:
final_df

,Date,Name,Team,Position,Opp,Home/Away,Line,Moneyline,Projection,Diff,Pick,Result,Accuracy,over_under
0,04-05-2022,Jalen Green,HOU,G,BKN,@,24.50,-115,18.90,-5.60,24.5,TBD,Last 7 days Top 15 Accuracy: 39-50 (44%) Sea...,u
1,04-05-2022,Theo Maledon,OKC,G,POR,vs.,17.50,-113,12.60,-4.90,17.5,TBD,Last 7 days Top 15 Accuracy: 39-50 (44%) Sea...,u
2,04-05-2022,Kevin Love,CLE,F,ORL,@,11.50,-115,16.30,4.80,11.5,TBD,Last 7 days Top 15 Accuracy: 39-50 (44%) Sea...,o
3,04-05-2022,Keon Johnson,POR,G,OKC,@,13.50,-105,9.00,-4.50,13.5,TBD,Last 7 days Top 15 Accuracy: 39-50 (44%) Sea...,u
4,04-05-2022,Keldon Johnson,SAS,F,DEN,@,21.50,-135,17.60,-3.90,21.5,TBD,Last 7 days Top 15 Accuracy: 39-50 (44%) Sea...,u
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14867,10-19-2021,Grayson Allen,MIL,G,BKN,vs.,10.50,-113,11.23,0.73,10.5,TBD,Season-to-date Top 15 Accuracy: 1174-1140 (5...,o
14868,10-19-2021,Nemanja Bjelica,GSW,F,LAL,@,6.50,-120,6.87,0.37,6.5,TBD,Season-to-date Top 15 Accuracy: 1174-1140 (5...,o
14869,10-19-2021,DeAndre Jordan,LAL,C,GSW,vs.,4.50,-130,4.82,0.32,4.5,TBD,Season-to-date Top 15 Accuracy: 1174-1140 (5...,o
14870,10-19-2021,Pat Connaughton,MIL,G,BKN,vs.,7.50,-111,7.19,-0.31,7.5,TBD,Season-to-date Top 15 Accuracy: 1174-1140 (5...,u


In [29]:
final_df = final_df.copy()

In [30]:
final_df['Projection'] = final_df['Projection'].map(lambda x: x.rstrip('ou'))

In [31]:
accuracies = final_df.Accuracy.str.split(" ", expand = True)

In [32]:
final_df['seven_day_accuracy'] = accuracies[8]

In [33]:
final_df['seven_day_accuracy'] = final_df['seven_day_accuracy'].map(lambda x: x.lstrip('(').rstrip('%)'))

In [34]:
final_df.seven_day_accuracy = final_df.seven_day_accuracy.replace('', 50, regex=True)

In [35]:
final_df['seven_day_accuracy'] = pd.to_numeric(final_df['seven_day_accuracy'])

In [36]:
final_df['seven_day_accuracy'] = final_df['seven_day_accuracy'].map(lambda x: x/100)

In [37]:
final_df.drop(columns=['Accuracy'], inplace=True)

In [38]:
final_df['Date'] = pd.to_datetime(final_df['Date'])

In [39]:
# Final form betting df, additional cleaning and saving to csv below
sorted_df = final_df.sort_values(by=['Name', 'Date'])
sorted_df.reset_index(inplace=True, drop=True)

In [40]:
sorted_df['Line'] = pd.to_numeric(sorted_df.Line)

In [41]:
sorted_df = sorted_df.replace('EVEN', '+100')

In [42]:
sorted_df = sorted_df.replace('TBD', None)

In [43]:
sorted_df['Moneyline'] = pd.to_numeric(sorted_df.Moneyline)

In [44]:
sorted_df['Result'] = sorted_df.Result.apply(lambda x: str(x))

In [45]:
results = sorted_df.Result.str.split(" ", expand = True);results

,0
0,None
1,None
2,None
3,None
4,None
...,...
14867,None
14868,None
14869,None
14870,None


In [46]:
sorted_df.Result = results[3]

KeyError: 3

In [ ]:
sorted_df['Projection'] = pd.to_numeric(sorted_df.Projection)
sorted_df['Diff'] = pd.to_numeric(sorted_df.Diff)
sorted_df['Result'] = pd.to_numeric(sorted_df.Result)
sorted_df['Pick'] = pd.to_numeric(sorted_df.Pick)
sorted_df

In [ ]:
# Determine if website pick was a win or loss (for fun, potential spinoff angle).
conditions = [
    ((sorted_df['Result'] < sorted_df['Pick']) & (sorted_df['over_under'] == 'u')),
    ((sorted_df['Result'] < sorted_df['Pick']) & (sorted_df['over_under'] == 'o')),
    ((sorted_df['Result'] > sorted_df['Pick']) & (sorted_df['over_under'] =='o')),
    ((sorted_df['Result'] > sorted_df['Pick']) & (sorted_df['over_under'] == 'u')),
    (sorted_df['Result'].isna() == True)
    ]

# create a list of the values we want to assign for each condition
values = ['1', '0', '1', '0', np.nan]

# create a new column and use np.select to assign values to it using our lists as arguments
sorted_df['successful_pick'] = np.select(conditions, values)

In [ ]:
db = client['nba-data']

In [ ]:
collection = db['betting-data']

In [ ]:
collection.insert_many(sorted_df.to_dict('records'))

***

In [ ]:
%reset -fs

In [ ]:
import pandas as pd
import numpy as np
import time, os
from datetime import datetime
from pymongo import MongoClient
from dotenv import load_dotenv
from sportsipy.nba.boxscore import Boxscores
from sportsipy.nba.schedule import Boxscore
pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', None)

In [ ]:
load_dotenv()

In [ ]:
mongo_pw = os.environ.get("MONGO_PW")

In [ ]:
client = MongoClient('mongodb+srv://natedirenzo:' + mongo_pw + '@nba-db.grcwg.mongodb.net/myFirstDatabase?retryWrites=true&w=majority')


In [ ]:
db = client['nba-data']

In [ ]:
cursor = db['betting-data'].find()

In [ ]:
mongo_df = pd.DataFrame(list(cursor)); mongo_df

In [ ]:
games = Boxscores(datetime(2021, 10, 19), datetime(2022, 4, 2))


In [ ]:
games = games.games

In [ ]:
uri_list = []
for i in games.keys():
    for j in games[i]:
        uri_list.append(j['boxscore'])

In [ ]:
def get_data(uri):
    boxscore = Boxscore(uri)
    final_stats_df = pd.DataFrame()
    
    home_df = boxscore.home_players[0].dataframe
    for player in boxscore.home_players[1:]:
        home_df = pd.concat([home_df, player.dataframe], axis = 0)
    home_df['name'] = [x.name for x in boxscore.home_players]
    home_df
    
    away_df = boxscore.away_players[0].dataframe
    for player in boxscore.away_players[1:]:
        away_df = pd.concat([away_df, player.dataframe], axis = 0)
    away_df['name'] = [x.name for x in boxscore.away_players]
    
    player_stats = [home_df, away_df]
    stats_df = pd.concat(player_stats)
    pd.concat([final_stats_df, stats_df])
    return final_stats_df

    